In [1]:
from streetlevel import streetview
from config import Config
from src.inference.segment import detect_trees
from src.inference.depth import estimate_depth
from src.utils.unwrap import divide_panorama
from src.utils.masks import add_masks, remove_duplicates, make_image
from src.utils.transformation import get_point
from src.utils.geodesic import get_coordinates
from cli import build_config
from models.DepthAnything.depth_anything_v2.dpt import DepthAnythingV2
from models.CalibrateDepth.model import DepthCalibrator
from config import Config
import torch
from ultralytics import YOLO
import pandas as pd
import folium
import os
import cv2
import numpy as np
import nest_asyncio
from IPython.display import display
import matplotlib.pyplot as plt
from src.utils.geodesic import localize_pixel_with_depth
from concurrent.futures import ThreadPoolExecutor
IO_EXECUTOR = ThreadPoolExecutor(max_workers=4)


xFormers not available
xFormers not available


In [2]:
def _fetch_pano_image(pano_id: str):
    nest_asyncio.apply()
    pano = streetview.find_panorama_by_id(pano_id, download_depth=False)
    if pano is None:
        raise ValueError(f"Panorama not found: {pano_id}")
    return pano

In [3]:
pano_id = "6YbSOJYq_bGE0E83-JFA2A"
pano_point = (5158, 3461)
pano = _fetch_pano_image(pano_id)
neighbor_1_id = "zXNDa-3To6LUVVwCusNXAg"
neighbor_1_point = (3195, 3694)
neighbor_1 = _fetch_pano_image(neighbor_1_id)
neighbor_2_id = "NaIDqjzbG2c_etjVwaUo6w"
neighbor_2_point = (5724, 3361)
neighbor_2 = _fetch_pano_image(neighbor_2_id)

In [4]:
# ---------- Step 1: Lat/Lon/Elev  --> local East-North-Up (meters) ----------
import numpy as np
from dataclasses import dataclass

# WGS84 constants
_A  = 6378137.0               # semi-major axis [m]
_F  = 1 / 298.257223563
_E2 = _F * (2 - _F)           # eccentricity^2

def geodetic_to_ecef(lat, lon, h):
    """lat/lon in radians, h in meters -> ECEF (X,Y,Z) in meters"""
    slat, clat = np.sin(lat), np.cos(lat)
    slon, clon = np.sin(lon), np.cos(lon)
    N = _A / np.sqrt(1 - _E2 * slat**2)
    X = (N + h) * clat * clon
    Y = (N + h) * clat * slon
    Z = (N * (1 - _E2) + h) * slat
    return np.array([X, Y, Z], float)

def ecef_to_enu_matrix(lat0, lon0):
    """Rotation matrix that maps ECEF delta -> ENU at reference (lat0, lon0)."""
    slat, clat = np.sin(lat0), np.cos(lat0)
    slon, clon = np.sin(lon0), np.cos(lon0)
    # Rows are E, N, U in ECEF basis
    E = np.array([-slon,            clon,           0.0])
    N = np.array([-slat*clon, -slat*slon,  clat])
    U = np.array([ clat*clon,  clat*slon,  slat])
    return np.vstack([E, N, U])

@dataclass
class LocalFrame:
    lat0_rad: float
    lon0_rad: float
    h0_m: float
    X0: np.ndarray     # ECEF of origin (3,)
    R_ecef2enu: np.ndarray  # 3x3

def make_local_frame(lat0_deg, lon0_deg, h0_m=0.0):
    lat0 = np.deg2rad(lat0_deg); lon0 = np.deg2rad(lon0_deg)
    X0   = geodetic_to_ecef(lat0, lon0, h0_m)
    R    = ecef_to_enu_matrix(lat0, lon0)
    return LocalFrame(lat0, lon0, h0_m, X0, R)

def lla_to_enu(lat_deg, lon_deg, h_m, lf: LocalFrame):
    """Convert any (lat,lon,h) to local ENU meters wrt lf origin."""
    X = geodetic_to_ecef(np.deg2rad(lat_deg), np.deg2rad(lon_deg), h_m)
    dX = X - lf.X0
    return lf.R_ecef2enu @ dX  # (E, N, U)


In [5]:
# ---------- Step 2: Pixel (u,v) -> world direction (unit vector in ENU) ----------
def pixel_to_cam_dir(u, v, w, h):
    """Equirect mapping -> unit direction in the camera's native axes.
       Convention: center column points 'forward' (azimuth 0), top=+elevation.
    """
    theta = 2*np.pi * (u / w - 0.5)     # azimuth in camera frame (-pi..pi), 0 at center column
    phi   =  np.pi * (0.5 - v / h)      # elevation (-pi/2..pi/2), +up
    d_cam = np.array([
        np.cos(phi) * np.sin(theta),    # x_cam
        np.sin(phi),                    # y_cam (up)
        np.cos(phi) * np.cos(theta)     # z_cam (forward)
    ], float)
    return d_cam / np.linalg.norm(d_cam)

def Rz(angle):
    c, s = np.cos(angle), np.sin(angle)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], float)

def Rx(angle):
    c, s = np.cos(angle), np.sin(angle)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]], float)

def cam_dir_to_world(d_cam, heading_rad, pitch_rad=0.0, roll_rad=0.0):
    """Rotate camera-direction into world ENU using pano heading (yaw), plus small pitch/roll."""
    # Heading is yaw around Up (world Z = Up in ENU). Pitch is small 'tilt'.
    R_world_from_cam = Rz(heading_rad) @ Rx(pitch_rad) @ Rz(roll_rad)
    d_world = R_world_from_cam @ d_cam
    return d_world / np.linalg.norm(d_world)


In [6]:
# ---------- Step 3: Build a small camera record from a StreetViewPanorama ----------
from dataclasses import dataclass

@dataclass
class Camera:
    pano_id: str
    C_enu: np.ndarray   # camera center in ENU meters, shape (3,)
    heading: float      # radians
    pitch: float        # radians
    roll: float         # radians
    w: int              # image width at chosen zoom
    h: int              # image height at chosen zoom
    depth: object | None  # pano.depth (DepthMap) or None

def build_camera_from_pano(pano, lf: LocalFrame, zoom=5, default_alt=0.0):
    # Position (meters in local ENU)
    alt = pano.elevation if getattr(pano, "elevation", None) is not None else default_alt
    C_enu = lla_to_enu(pano.lat, pano.lon, alt, lf)

    # Image size at chosen zoom
    if getattr(pano, "image_sizes", None) and len(pano.image_sizes) > zoom:
        size = pano.image_sizes[zoom]      # Size(x=..., y=...)
        w, h = int(size.x), int(size.y)
    else:
        from streetlevel import streetview
        img = streetview.get_panorama(pano, zoom=zoom)  # PIL
        w, h = img.size

    return Camera(
        pano_id=pano.id,
        C_enu=C_enu,
        heading=(pano.heading or 0.0),
        pitch=(pano.pitch or 0.0),
        roll=(pano.roll or 0.0),
        w=w, h=h,
        depth=getattr(pano, "depth", None)
    )


In [7]:
def pixel_ray_and_depth(cam: Camera, u, v, depth_is_horizontally_flipped=True):
    d_cam   = pixel_to_cam_dir(u, v, cam.w, cam.h)
    d_world = cam_dir_to_world(d_cam, cam.heading, cam.pitch, cam.roll)

    s = None
    if cam.depth is not None and getattr(cam.depth, "data", None) is not None:
        Hd, Wd = cam.depth.data.shape[:2]   # (height, width)
        ud = int((1 - u / cam.w) * Wd) if depth_is_horizontally_flipped else int((u / cam.w) * Wd)
        vd = int((v / cam.h) * Hd)
        ud = np.clip(ud, 0, Wd-1); vd = np.clip(vd, 0, Hd-1)
        depth_val = float(cam.depth.data[vd, ud])  # meters; -1 => invalid/horizon
        if depth_val >= 0:
            s = depth_val

    return cam.C_enu, d_world / np.linalg.norm(d_world), s


In [8]:
from streetlevel import streetview

# 0) Fetch panorama OBJECTS with depth/meta (not just images)
pano       = streetview.find_panorama_by_id("6YbSOJYq_bGE0E83-JFA2A", download_depth=True)
neighbor_1 = streetview.find_panorama_by_id("zXNDa-3To6LUVVwCusNXAg", download_depth=True)
neighbor_2 = streetview.find_panorama_by_id("NaIDqjzbG2c_etjVwaUo6w", download_depth=True)

# 1) Build the local ENU frame from the first pano
lf = make_local_frame(pano.lat, pano.lon, h0_m=(pano.elevation or 0.0))

# 2) Use the SAME zoom you used when picking pixels.
#    Default in streetlevel is 5, so:
ZOOM = 5

# 3) Build camera records
camA = build_camera_from_pano(pano,       lf, zoom=ZOOM)
camB = build_camera_from_pano(neighbor_1, lf, zoom=ZOOM)
camC = build_camera_from_pano(neighbor_2, lf, zoom=ZOOM)

# 4) Your pixel picks (u,v)
pano_point        = (4293, 3744)
neighbor_1_point = (1631, 3727)
neighbor_2_point = (5425, 3544)

# 5) Rays (+ optional low-res depth in meters)
DEPTH_FLIPPED = True  # flip to True only if you’ve confirmed a horizontal flip in your depth map
C_A, d_A, s_A = pixel_ray_and_depth(camA, *pano_point, depth_is_horizontally_flipped=DEPTH_FLIPPED)
C_B, d_B, s_B = pixel_ray_and_depth(camB, *neighbor_1_point, depth_is_horizontally_flipped=DEPTH_FLIPPED)
C_C, d_C, s_C = pixel_ray_and_depth(camC, *neighbor_2_point, depth_is_horizontally_flipped=DEPTH_FLIPPED)
print("A:", C_A, d_A, s_A)
print("B:", C_B, d_B, s_B)
print("C:", C_C, d_C, s_C)


A: [          0           0           0] [    -0.8163      0.3849     0.43071] 9.019837041025484
B: [    -8.0359      9.0033     0.15862] [   -0.65948     0.26384     -0.7039] 10.164468616189557
C: [     7.2601     -8.1503   0.0053617] [   -0.49288     0.25658     0.83141] 17.00810261420435


In [9]:
seed_range = s_A if s_A is not None else 20.0  # meters (any plausible number works as a seed)
X_seed = C_A + seed_range * d_A                 # a guessed 3D location for the clicked point

In [10]:
def world_point_to_uv(cam, X):
    # rotate world->camera, then convert direction to equirectangular pixels
    R = Rz(cam.heading) @ Rx(cam.pitch) @ Rz(cam.roll)
    d = (X - cam.C_enu)
    d = d / np.linalg.norm(d)
    dc = R.T @ d
    theta = np.arctan2(dc[0], dc[2])        # azimuth
    phi   = np.arcsin(dc[1])                # elevation
    u = cam.w * (theta/(2*np.pi) + 0.5)
    v = cam.h * (0.5 - phi/np.pi)
    return u, v

uB_pred, vB_pred = world_point_to_uv(camB, X_seed)

In [11]:
def pair_metrics(C1, d1, C2, d2, eps=1e-9):
    # returns along-ray distances t1,t2 to closest approach, and the gap (meters)
    d1 = d1/np.linalg.norm(d1); d2 = d2/np.linalg.norm(d2)
    b  = float(np.dot(d1, d2))
    w0 = C1 - C2
    d  = float(np.dot(d1, w0))
    e  = float(np.dot(d2, w0))
    den = 1.0 - b*b
    if abs(den) < eps: return None
    t1 = (b*e - d) / den
    t2 = (e - b*d) / den
    P1 = C1 + t1*d1; P2 = C2 + t2*d2
    gap = np.linalg.norm(P1 - P2)
    return t1, t2, gap

def refine_uv_against_ref(cam, u0, v0, C_ref, d_ref, search_px=40, step_px=2, require_positive_t=True):
    best = None
    for dv in range(-search_px, search_px+1, step_px):
        v = v0 + dv
        if v < 0 or v >= cam.h: continue
        for du in range(-search_px, search_px+1, step_px):
            u = u0 + du
            if u < 0 or u >= cam.w: continue
            _, d_world, _ = pixel_ray_and_depth(cam, u, v)
            met = pair_metrics(cam.C_enu, d_world, C_ref, d_ref)
            if met is None: 
                continue
            t_self, t_ref, gap = met
            if require_positive_t and (t_self <= 0 or t_ref <= 0):
                continue
            if (best is None) or (gap < best[0]):
                best = (gap, u, v, t_self, t_ref, d_world)
    return best  # (gap, u*, v*, t_self, t_ref, d_world*) or None

bestB = refine_uv_against_ref(camB, uB_pred, vB_pred, C_ref=C_A, d_ref=d_A)
print("Refined in B:", bestB[:5] if bestB else None)


Refined in B: (np.float64(2.432376777795247e-15), np.float64(8379.194534892473), np.float64(4684.406139231335), 6.7035010497324485, 9.01983704102548)


In [12]:
def triangulate_from_rays(Cs, ds):
    # least-squares point closest to all rays
    A = np.zeros((3,3)); b = np.zeros(3)
    for C, d in zip(Cs, ds):
        d = d/np.linalg.norm(d)
        P = np.eye(3) - np.outer(d, d)
        A += P; b += P @ C
    X = np.linalg.solve(A, b)
    # residual distances (meters) from X to each ray (smaller is better)
    res = [np.linalg.norm(np.cross((X - C), d/np.linalg.norm(d))) for C, d in zip(Cs, ds)]
    return X, np.array(res)

gap,uB,vB,tB,tA,dB_star = bestB
X, residuals = triangulate_from_rays([C_A, C_B], [d_A, dB_star])
print("Triangulated point X (meters, ENU):", X)
print("Residuals to rays (m):", residuals)  # aim ≲ ~1 m each


Triangulated point X (meters, ENU): [    -7.3629      3.4717      3.8849]
Residuals to rays (m): [ 1.1957e-15  2.0237e-15]


In [13]:
uC_pred, vC_pred = world_point_to_uv(camC, X)
bestC = refine_uv_against_ref(camC, uC_pred, vC_pred, C_ref=C_A, d_ref=d_A)
if bestC:
    _,uC,vC,tC,tA2,dC_star = bestC
    X, residuals = triangulate_from_rays([C_A, C_B, C_C], [d_A, dB_star, dC_star])
    print("Refined X with C:", X, "residuals:", residuals)

Refined X with C: [    -7.3629      3.4717      3.8849] residuals: [ 3.8202e-15  1.1763e-15  3.9968e-15]


In [14]:
from pyproj import CRS, Transformer

def enu_to_lla(X_enu, lf):
    ecef = CRS.from_epsg(4978); lla = CRS.from_epsg(4979)
    R_enu2ecef = lf.R_ecef2enu.T
    X_ecef = lf.X0 + R_enu2ecef @ X_enu
    to_lla = Transformer.from_crs(ecef, lla, always_xy=True)
    lon, lat, h = to_lla.transform(X_ecef[0], X_ecef[1], X_ecef[2])
    return lat, lon, h

lat, lon, h = enu_to_lla(np.array([-7.3629, 3.4717, 3.8849]), lf)
print("Lat/Lon/Height:", lat, lon, h)


Lat/Lon/Height: 30.7113450674645 76.80049285916941 361.3107169615105


In [15]:
def world_point_to_uv(cam, X):
    R = Rz(cam.heading) @ Rx(cam.pitch) @ Rz(cam.roll)
    d = (X - cam.C_enu); d /= np.linalg.norm(d)
    dc = R.T @ d
    theta = np.arctan2(dc[0], dc[2]); phi = np.arcsin(dc[1])
    u = cam.w * (theta/(2*np.pi) + 0.5)
    v = cam.h * (0.5 - phi/np.pi)
    return u, v

print("A proj:", world_point_to_uv(camA, X))
print("B proj:", world_point_to_uv(camB, X))
print("C proj:", world_point_to_uv(camC, X))

A proj: (np.float64(4293.000000000001), np.float64(3743.999999999999))
B proj: (np.float64(8379.194534892473), np.float64(4684.406139231334))
C proj: (np.float64(3761.761481629452), np.float64(3317.3788278704005))


In [16]:
nest_asyncio.apply()
image = streetview.get_panorama(pano)
image = np.array(image)
W = image.shape[1]
H = image.shape[0]

pano_result = localize_pixel_with_depth(
    pano, 4293, 3744, W, H, 9.019837041025484,
    depth_is_slant=True,        # Street View depth is a slant range
    snap_to_ground=False        # set True if you want a ground footprint
)
neighbor_1_result = localize_pixel_with_depth(
    pano, 1631, 3727, W, H, 10.164468616189557,
    depth_is_slant=True,        # Street View depth is a slant range
    snap_to_ground=False        # set True if you want a ground footprint
)
neighbor_2_result = localize_pixel_with_depth(
    pano, 5425, 3544, W, H, 17.00810261420435,
    depth_is_slant=True,        # Street View depth is a slant range
    snap_to_ground=False        # set True if you want a ground footprint
)

In [20]:
pano_result, neighbor_1_result, neighbor_2_result, (lat, lon)

((30.71134506753596, 76.80049285936016),
 (30.711338877307195, 76.80050043245336),
 (30.711352387467667, 76.80048171221509),
 (30.7113450674645, 76.80049285916941))

In [19]:
import folium

def plot_panos_on_map():
    """
    Plots the given pano and its two nearest neighbors on a folium map.
    """
    fmap = folium.Map(location=[pano.lat, pano.lon], zoom_start=18)

    # Add marker for the main pano
    folium.CircleMarker(
        [pano_result[0], pano_result[1]],
        popup=f"Main Pano",
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        radius=0.5
    ).add_to(fmap)

    folium.CircleMarker(
        [neighbor_1_result[0], neighbor_1_result[1]],
        popup=f"Neighbor1",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        radius=0.5
    ).add_to(fmap)

    folium.CircleMarker(
        [neighbor_2_result[0], neighbor_2_result[1]],
        popup=f"Neighbor2",
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        radius=0.5
    ).add_to(fmap)

    folium.CircleMarker(
        [lat, lon],
        popup=f"Main Pano",
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7,
        radius=0.5
    ).add_to(fmap)

    folium.CircleMarker(
        [pano.lat, pano.lon],
        popup=f"Main Pano",
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        radius=3
    ).add_to(fmap)

    folium.CircleMarker(
        [neighbor_1.lat, neighbor_1.lon],
        popup=f"Neighbor1",
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        radius=3
    ).add_to(fmap)

    folium.CircleMarker(
        [neighbor_2.lat, neighbor_2.lon],
        popup=f"Neighbor2",
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        radius=3
    ).add_to(fmap)

    return fmap

fmap = plot_panos_on_map()
import folium

# Add Esri WorldImagery tile layer to the map
esri_layer = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri WorldImagery',
    overlay=True,
    control=True,
    max_zoom=25
)
esri_layer.add_to(fmap)

display(fmap)
